In [66]:
#!/usr/bin/env python

# convert gml to json and add community grouping

# TODO: fix relative filepaths for file input and export

# import libraries
# import argparse
import json
import networkx as nx
from networkx.readwrite import json_graph
from modularity_maximization import partition
from modularity_maximization.utils import get_modularity


def gmltojson(gmlfile, jsonfile):
    """
    Converts GML file to json while adding communities/modularity groups
    using modularity_maximization. JSON output is usable with D3 force layout.
    Usage:
    python convert.py -i mygraph.graphml -o outfile.json
    Return data in node-link format that is suitable for JSON serialization and use in Javascript documents.
    """

    print('Reading file:', gmlfile)
    di_graph = nx.read_gml('../data/raw/' + gmlfile)
    comm_dict = partition(di_graph)
    print('Calculating communities...')
    for comm in set(comm_dict.values()):
        print("Community %d" % comm)
        print(', '.join([node for node in comm_dict if comm_dict[node] == comm]))

    print('Modularity of such partition for network is %.3f' % \
          get_modularity(di_graph, comm_dict))

    # adds partition/community number as attribute named 'Modularity Class'
    print('Assigning Communities...')
    for n, d in di_graph.nodes(data=True):
        d['Modularity Class'] = comm_dict[n]
        
    # set positions of nodes
    pos = nx.spring_layout(di_graph)

    for node,(x,y) in pos.items():
        di_graph.node[node]['x'] = float(x)
        di_graph.node[node]['y'] = float(y)

    # TODO: create betweeness centrality and degrees, degree centrality, in-degree etc
    bb = nx.betweenness_centrality(di_graph)
    nx.set_node_attributes(di_graph, name='betweeness cent.', values=bb)

    # create a dictionary in a node-link format that is suitable for JSON serialization
    with open('../data/processed/' + jsonfile + '.json', 'w') as outfile1:
        outfile1.write(json.dumps(json_graph.node_link_data(G=di_graph)))
    print('Complete!')
    
    # see: https://cambridge-intelligence.com/keylines-faqs-social-network-analysis/



In [67]:
gmltojson('TheDataFox.gml', 'TheDataFox')

Reading file TheDataFox.gml
Calculating communities...
Community 5
NVIDIAGeForce, OmnicLab, AMDGaming, TESLegends, OverwatchEU, engadget, msiUSA, OWCentral, techradar, PlayDuelyst, ASUS, BlizzardCSEU_EN, overwatchleague, OverwatchGG, PCPartPicker, pcgamer, PlayOverwatch, nvidia, NVIDIAGeForceUK, Twitch, PocketPixl, PcCentric, TEAMEVGA, EGX_Esports, Radeon, PlayHearthstone, YourOverwatchYT, GIGABYTEUSA, CORSAIR, tomshardware, ClanTDK, NETDUMA
Community 7
mangacom, ViralRajyaguru, MangaUK, metline, DB_Legends, FUNimation, SonSonComics, savingoceans, GoodSmile_US, animetoday, SonSonPhotos, VIZMedia, Crunchyroll
Community 8
Bulbagarden, SixPrizes, SerebiiNet, PokemonUKGo, Junichi_Masuda, pokebeach_wpm, NintendoUK, TeamMysticPGO, Bulbanews, GamingHistorian, OmniPoke, PokemonGoNews, BulbaNewsNOW, Bulbapedia, enjoifriend, RareCandyTCG, NinEverything, PokemonNewsUK, Pokemon_cojp, theSuperRodcast, nintendolife, GoTeamMystic, PokemonGoApp, capsulecorpccg, NintendoAmerica, Pokemon, ThePokemonEvos